## FASE 2

### Contar desbalance de clases

In [2]:
import os
import polars as pl
from collections import defaultdict

# Ruta de la carpeta que contiene los archivos CSV
ruta_carpeta = "C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/DATASET ACTUALIZADO V2"

if not os.path.exists(ruta_carpeta):
    raise Exception(f"La carpeta {ruta_carpeta} no existe.")

archivos_csv = [os.path.join(ruta_carpeta, f) for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
conteo_total = {}

# Columnas conflictivas conocidas (puedes agregar más si aparece otro error)
columnas_conflictivas = [
    "TotLen Bwd Pkts", "Active Mean", "Active Std", "Dst Port"
]

# Crear un dict para forzar tipos como Float64
schema_overrides = {col: pl.Float64 for col in columnas_conflictivas}

# Parámetros de chunk
chunk_size = 300_000

for archivo in archivos_csv:
    try:
        print(f"Procesando archivo {archivo}...")
        
        # Leemos el archivo en "lazy" para no cargar todo en memoria
        lazy_df = pl.scan_csv(
            archivo,
            infer_schema_length=10000,  # mayor para detectar correctamente los tipos
            schema_overrides=schema_overrides,
            ignore_errors=True          # omite filas con errores
        )

        # Obtenemos el número total de filas del archivo
        total_rows = lazy_df.select(pl.count()).collect().item()

        if total_rows == 0:
            print(f"  El archivo {os.path.basename(archivo)} está vacío.")
            continue

        # Procesamos el archivo por chunks
        for start_row in range(0, total_rows, chunk_size):
            df_chunk = lazy_df.slice(start_row, chunk_size).collect()
            if df_chunk.is_empty():
                break

            # Procesamos las etiquetas si existen
            if "Label" in df_chunk.columns:
                labels_count = (
                    df_chunk.group_by("Label")  # agrupamos por la columna 'Label'
                    .agg(pl.count().alias("count"))
                    .sort("Label")
                )

                for row in labels_count.iter_rows():
                    label, count = row
                    conteo_total[label] = conteo_total.get(label, 0) + count
            else:
                print(f"  La columna 'Label' no se encuentra en el archivo {os.path.basename(archivo)}")

    except Exception as e:
        print(f"Error al procesar el archivo {archivo}:\n  {e}")

# Mostrar conteo total
print("\nConteo total de etiquetas en todos los archivos:")
for label in sorted(conteo_total.keys()):
    print(f"  Etiqueta {label}: {conteo_total[label]} veces")

Procesando archivo C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/DATASET ACTUALIZADO V2\Friday-02-03-2018.csv...


C:\Users\josue\AppData\Local\Temp\ipykernel_18104\2719318449.py:38: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows = lazy_df.select(pl.count()).collect().item()
C:\Users\josue\AppData\Local\Temp\ipykernel_18104\2719318449.py:54: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("count"))


Procesando archivo C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/DATASET ACTUALIZADO V2\Friday-23-02-2018.csv...
Procesando archivo C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/DATASET ACTUALIZADO V2\Thursday-01-03-2018.csv...
Procesando archivo C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/DATASET ACTUALIZADO V2\Thursday-22-02-2018.csv...
Procesando archivo C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/DATASET ACTUALIZADO V2\Wednesday-28-02-2018.csv...

Conteo total de etiquetas en todos los archivos:
  Etiqueta BENIGN: 31245820 veces
  Etiqueta Botnet Ares: 142921 veces
  Etiqueta Botnet Ares - Attempted: 262 veces
  Etiqueta Infiltration - Communication Victim Attacker: 204 veces
  Etiqueta Infiltration - Dropbox Download: 85 veces
  Etiqueta Infiltration - Dropbox Download - Attempted: 28 veces
  Etiqueta Infiltration - NMAP Portscan: 89374 veces
  Etiqueta Web Attack - Brut

### limitar la clase bening 

In [ ]:
import os
import polars as pl

# Carpetas
input_folder = r"C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/DATASET ACTUALIZADO V2/"
output_folder = r"C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/dataset_undersampling/"

os.makedirs(output_folder, exist_ok=True)

# Parámetros
limite_benign_total = 500_000
benign_por_archivo = 100_000
benign_acumulado = 0

archivos_csv = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

for archivo in archivos_csv:
    print(f"Procesando {archivo}...")

    # Leer archivo completo (Eager)
    df = pl.read_csv(os.path.join(input_folder, archivo))

    # Separar BENIGN y resto
    benign_df = df.filter(pl.col("Label") == "BENIGN")
    resto_df = df.filter(pl.col("Label") != "BENIGN")

    # Limitar BENIGN a 60k por archivo, respetando límite global
    if benign_acumulado < limite_benign_total:
        remaining_global = limite_benign_total - benign_acumulado
        take = min(benign_por_archivo, remaining_global, benign_df.shape[0])
        # Selección aleatoria de registros de la clase BENIGN
        benign_keep = benign_df.sample(n=take, shuffle=True, seed=42) 
        benign_acumulado += benign_keep.shape[0]
    else:
        benign_keep = pl.DataFrame()

    # Combinar BENIGN limitado + resto
    df_final = pl.concat([benign_keep, resto_df], how="diagonal_relaxed")

    # Guardar archivo procesado en la nueva carpeta
    output_path = os.path.join(output_folder, archivo)
    df_final.write_csv(output_path)
    print(f"Guardado en: {output_path}")
    print(f"BENIGN acumulado hasta ahora: {benign_acumulado}")
    print(f"Filas procesadas de este archivo: {df_final.shape[0]}")

    # Liberar memoria antes de procesar el siguiente archivo
    del df, benign_df, resto_df, benign_keep, df_final

    # Si se alcanzó el límite global de BENIGN, avisar
    if benign_acumulado >= limite_benign_total:
        print(" Límite global de BENIGN alcanzado.")


### Correlaciones iniciales

In [ ]:
import os
import polars as pl
import matplotlib.pyplot as plt
import numpy as np

# Ruta de tus archivos
ruta_carpeta = r"C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/dataset_undersampling/"

if not os.path.exists(ruta_carpeta):
    raise Exception(f"La carpeta {ruta_carpeta} no existe.")

archivos_csv = [os.path.join(ruta_carpeta, f) for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]

# Cargar todos los CSV directamente en memoria
dfs = []
for archivo in archivos_csv:
    print(f"🔹 Cargando {archivo}...")
    df_temp = pl.read_csv(
        archivo,
        infer_schema_length=50000,
        ignore_errors=True
    )
    dfs.append(df_temp)

#  Concatenar todos los archivos
df = pl.concat(dfs, how="diagonal_relaxed")
df = pl.DataFrame(df)
del dfs

print("\n Datos cargados correctamente (sin límite en Benign)")
print("Número de filas:", df.shape[0])
print("Número de columnas:", df.shape[1])

#  Normalizar etiquetas
label_mapping = {
    "Botnet Ares": "Bot",
    "Web Attack - Brute Force": "WebAttacks",
    "Web Attack - XSS": "WebAttacks",
    "Infiltration - Communication Victim Attacker": "Infilteration",
    "Infiltration - Dropbox Download": "Infilteration",
    "Infiltration - NMAP Portscan": "Infilteration",
}

df = df.with_columns([
    pl.when(pl.col("Label").str.to_lowercase() == "benign")
      .then(pl.lit("benign"))
      .when(pl.col("Label").str.contains("Attempted"))
      .then(pl.lit("benign"))
      .when(pl.col("Label").is_in(list(label_mapping.keys())))
      .then(
          pl.col("Label").replace_strict(
              label_mapping, 
              return_dtype=pl.Utf8,
              default="Other"   # <- valores no mapeados van a "Other"
          )
      )
      .otherwise(pl.col("Label"))
      .alias("Label")
])



#  Eliminar etiquetas con "SQL"
df = df.filter(~pl.col("Label").str.contains("SQL"))

#  Conteo de etiquetas después de limpieza
conteo_etiquetas = df.group_by("Label").agg(pl.count().alias("count")).sort("count", descending=True)
print("\n Distribución de clases finales:\n", conteo_etiquetas)

# 🔧 Convertir columnas numéricas correctamente (excluyendo Label y campos no numéricos)
excluir = ['id', 'Flow ID','Protocol',  'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp', 'Label']
for col in df.columns:
    if col not in excluir:
        df = df.with_columns([
            pl.col(col)
            .cast(pl.Utf8)
            .str.replace(",", "")
            .str.replace(r"[Ee]\+?", "e", literal=False)
            .cast(pl.Float64, strict=False)
        ])



#  Mapa de calor de correlaciones con columnas numéricas
num_cols = [c for c in df.columns if df[c].dtype == pl.Float64]
if len(num_cols) > 1:
    corr = df.select(num_cols).to_pandas().corr()
    plt.figure(figsize=(14, 10))
    plt.imshow(corr, cmap="coolwarm", interpolation="nearest")
    plt.colorbar()
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90, fontsize=5)
    plt.yticks(range(len(corr.columns)), corr.columns, fontsize=5)
    plt.title("Mapa de calor de correlaciones", fontsize=12)
    plt.tight_layout()
    plt.show()

#  Gráficas de solo 2 features y 4 etiquetas → 8 subplots
features = ["Flow Duration", "Flow Packets/s"]
orden_personal = ["benign","Bot", "Infilteration", "WebAttacks"]
labels_finales = [lab for lab in orden_personal if lab in df["Label"].unique().to_list()]


fig, axes = plt.subplots(2, len(labels_finales), figsize=(18, 8))
axes = axes.flatten()

for row, feature in enumerate(features):
    for col, etiqueta in enumerate(labels_finales):
        ax = axes[row * len(labels_finales) + col]
        subset = df.filter(pl.col("Label") == etiqueta)
        if subset.shape[0] > 0:
            n = min(100000, subset.shape[0])
            subset_sample = subset.sample(n=n, seed=42)

            valores = np.array(subset_sample[feature].to_list(), dtype=np.float64)
            valores = valores[np.isfinite(valores)]

            if len(valores) > 0:
                ax.hist(valores, bins=50, alpha=0.7, color="steelblue")

        ax.set_title(f"{etiqueta} - {feature}")
        ax.set_xlabel("Valor")
        ax.set_ylabel("Frecuencia")

plt.tight_layout()
plt.show()



#  Nueva figura para 2 features → 8 subplots
features_nuevas = ["Flow Bytes/s", "Average Packet Size"]
orden_personal = ["benign","Bot", "Infilteration", "WebAttacks"]
labels_finales = [lab for lab in orden_personal if lab in df["Label"].unique().to_list()]

fig, axes = plt.subplots(2, len(labels_finales), figsize=(18, 8))
axes = axes.flatten()

for row, feature in enumerate(features_nuevas):
    for col, etiqueta in enumerate(labels_finales):
        ax = axes[row * len(labels_finales) + col]
        subset = df.filter(pl.col("Label") == etiqueta)
        if subset.shape[0] > 0:
            n = min(50000, subset.shape[0])
            subset_sample = subset.sample(n=n, seed=42)

            valores = np.array(subset_sample[feature].to_list(), dtype=np.float64)
            valores = valores[np.isfinite(valores)]  # elimina NaN, inf, -inf

            if len(valores) > 0:
                ax.hist(valores, bins=50, alpha=0.7, color="steelblue")

        ax.set_title(f"{etiqueta} - {feature}", fontsize=9)
        ax.set_xlabel("Valor", fontsize=8)
        ax.set_ylabel("Frecuencia", fontsize=8)

plt.tight_layout()
plt.show()


### Verficar valores NaN, Nulos y duplicados

In [1]:
import os
import polars as pl
import numpy as np

# Ruta de la carpeta con los CSV
ruta_carpeta = r"C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/dataset_undersampling/"

if not os.path.exists(ruta_carpeta):
    raise Exception(f"La carpeta {ruta_carpeta} no existe.")

archivos_csv = [os.path.join(ruta_carpeta, f) for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]

# Columnas a excluir
excluir = ['id', 'Flow ID', 'Protocol', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp', 'Label']

# Procesar cada archivo
for archivo in archivos_csv:
    print(f"\n Analizando archivo: {os.path.basename(archivo)}")

    # Cargar dataset
    df = pl.read_csv(archivo)

    # Asegurar que las columnas excluidas existen en el DataFrame
    cols_excluir = [c for c in excluir if c in df.columns]

    # Seleccionar columnas numéricas (excluyendo las de 'excluir')
    cols_numericas = [c for c in df.columns if c not in cols_excluir]

    # Convertir a Float64
    df = df.with_columns([df[c].cast(pl.Float64, strict=False) for c in cols_numericas])

    # Total de filas
    total_filas = df.shape[0]

    # Contar NaN
    nan_count = df[cols_numericas].null_count().to_pandas().sum().sum()

    # Contar Inf y -Inf
    df_pandas = df[cols_numericas].to_pandas()
    inf_count = df_pandas.replace([np.inf, -np.inf], np.nan).isna().sum().sum() - nan_count

    # Contar duplicados (filas exactamente iguales)
    duplicados = df.is_duplicated().sum()

    print(f"   Total de filas: {total_filas}")
    print(f"   Valores NaN: {nan_count}")
    print(f"   Valores Inf/-Inf: {inf_count}")
    print(f"   Filas duplicadas: {duplicados}")



 Analizando archivo: Friday-02-03-2018.csv
   Total de filas: 243183
   Valores NaN: 0
   Valores Inf/-Inf: 0.0
   Filas duplicadas: 0

 Analizando archivo: Friday-23-02-2018.csv
   Total de filas: 100230
   Valores NaN: 0
   Valores Inf/-Inf: 0.0
   Filas duplicadas: 0

 Analizando archivo: Thursday-01-03-2018.csv
   Total de filas: 139847
   Valores NaN: 0
   Valores Inf/-Inf: 0.0
   Filas duplicadas: 0

 Analizando archivo: Thursday-22-02-2018.csv
   Total de filas: 100208
   Valores NaN: 0
   Valores Inf/-Inf: 0.0
   Filas duplicadas: 0

 Analizando archivo: Wednesday-28-02-2018.csv
   Total de filas: 149844
   Valores NaN: 0
   Valores Inf/-Inf: 0.0
   Filas duplicadas: 0


## FASE 3

### Verificar los valores constantes en columas

In [1]:
import os
import polars as pl
import numpy as np

# Ruta de la carpeta con los CSV
ruta_carpeta = r"C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/dataset_undersampling/"

if not os.path.exists(ruta_carpeta):
    raise Exception(f"La carpeta {ruta_carpeta} no existe.")

archivos_csv = [os.path.join(ruta_carpeta, f) for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]

# Columnas a excluir
excluir = ['id', 'Flow ID', 'Protocol', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp', 'Label']

# Listas globales para comparar entre archivos
global_ceros = []
global_constantes = []

# Procesar cada archivo
for archivo in archivos_csv:
    print(f"\n Analizando archivo: {os.path.basename(archivo)}")

    # Cargar dataset
    df = pl.read_csv(archivo)

    # Asegurar que las columnas excluidas existen en el DataFrame
    cols_excluir = [c for c in excluir if c in df.columns]

    # Seleccionar columnas numéricas (excluyendo las de 'excluir')
    cols_numericas = [c for c in df.columns if c not in cols_excluir]

    # Convertir a Float64
    df = df.with_columns([df[c].cast(pl.Float64, strict=False) for c in cols_numericas])

    # Columnas con todos los valores en cero
    cols_ceros = [c for c in cols_numericas if df[c].sum() == 0]

    # Columnas constantes (un solo valor único en toda la columna)
    cols_constantes = [c for c in cols_numericas if df[c].n_unique() == 1]

    print(f"   Columnas con todos ceros: {cols_ceros}")
    print(f"   Columnas constantes: {cols_constantes}")

    # Guardar para análisis global
    global_ceros.append(set(cols_ceros))
    global_constantes.append(set(cols_constantes))

# ---- Análisis Global ----
if global_ceros:
    interseccion_ceros = set.intersection(*global_ceros)
else:
    interseccion_ceros = set()

if global_constantes:
    interseccion_constantes = set.intersection(*global_constantes)
else:
    interseccion_constantes = set()

print("\n ==== RESULTADO GLOBAL ====")
print(f"Columnas siempre ceros en TODOS los archivos: {sorted(interseccion_ceros)}")
print(f"Columnas siempre constantes en TODOS los archivos: {sorted(interseccion_constantes)}")



 Analizando archivo: Friday-02-03-2018.csv
   Columnas con todos ceros: ['Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count']
   Columnas constantes: ['Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count']

 Analizando archivo: Friday-23-02-2018.csv
   Columnas con todos ceros: ['Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count']
   Columnas constantes: ['Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count']

 Analizando archivo: Thursday-01-03-2018.csv
   Columnas con todos ceros: ['Bwd URG Flags']
   Columnas constantes: ['Bwd URG Flags']

 Analizando archivo: Thursday-22-02-2018.csv
   Columnas con todos ceros: ['Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count']
   Columnas constantes: ['Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count']

 Analizando archivo: Wednesday-28-02-2018.csv
   Columnas con todos ceros: ['Bwd URG Flags']
   Columnas constantes: ['Bwd URG Flags']

 ==== RESULTADO GLOBAL ====
Columnas siempre ceros en TODOS los archivos: ['Bwd URG Flags']
Columnas siempre constant

### Seleccion de caracteristicas

In [1]:
import pandas as pd
import polars as pl
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif, RFE
import warnings
import os

warnings.filterwarnings('ignore')

class CICIDSFeatureSelector:
    def __init__(self):
        self.df = None
        self.selected_features = None
        self.protocol_encoder = LabelEncoder()
        
    def load_data(self, df):
        """Cargar y preparar datos"""
        self.df = df.copy()
        
        # Reordenar: mover Label al final como columna target
        if 'Label' in self.df.columns:
            label_col = self.df.pop('Label')
            self.df['Label'] = label_col
            print(f"✓ Columna 'Label' movida al final como target")
        
        # Aplicar Label Encoding a la columna Protocol
        if 'Protocol' in self.df.columns:
            self.df['Protocol'] = self.protocol_encoder.fit_transform(self.df['Protocol'])
            print(f"✓ Label Encoding aplicado a columna 'Protocol'")
            print(f"  Protocolos únicos codificados: {len(self.protocol_encoder.classes_)}")
        
        print(f"✓ Datos cargados: {self.df.shape}")
        print(f"✓ Distribución de clases:")
        print(self.df.iloc[:, -1].value_counts())
        
    def exploratory_analysis(self):
        """Análisis exploratorio conciso"""
        print("\n📊 ANÁLISIS EXPLORATORIO")
        print("=" * 40)
        
        target_col = self.df.columns[-1]
        class_dist = self.df[target_col].value_counts()
        
        print(f"Shape: {self.df.shape}")
        print(f"Missing values: {self.df.isnull().sum().sum()}")
        print(f"Classes: {class_dist.to_dict()}")
        
        return class_dist
    
    def find_optimal_features(self, X, y, max_features=50):
        """Encontrar número óptimo de características automáticamente"""
        print("\n🔍 SELECCIÓN AUTOMÁTICA DE CARACTERÍSTICAS")
        print("=" * 45)
        
        # Codificar target
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)
        
        # Probar diferentes números de características usando validación cruzada
        feature_counts = range(10, min(max_features, X.shape[1]), 5)
        cv_scores = []
        
        print("🧪 Probando diferentes números de características...")
        
        rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
        
        for n_features in feature_counts:
            # Seleccionar características con F-test
            selector = SelectKBest(score_func=f_classif, k=n_features)
            X_selected = selector.fit_transform(X, y_encoded)
            
            # Validación cruzada
            scores = cross_val_score(rf, X_selected, y_encoded, cv=3, scoring='f1_weighted', n_jobs=-1)
            cv_scores.append(scores.mean())
            
            print(f"  {n_features:2d} features -> F1-Score: {scores.mean():.4f} (±{scores.std():.4f})")
        
        # Encontrar número óptimo
        optimal_idx = np.argmax(cv_scores)
        optimal_n_features = list(feature_counts)[optimal_idx]
        optimal_score = cv_scores[optimal_idx]
        
        print(f"\n🎯 NÚMERO ÓPTIMO: {optimal_n_features} características (F1: {optimal_score:.4f})")
        
        return optimal_n_features, feature_counts, cv_scores
    
    def feature_selection(self, method='hybrid'):
        """Selección de características con número óptimo automático"""
        print(f"\n⚡ SELECCIÓN DE CARACTERÍSTICAS ({method.upper()})")
        print("=" * 50)
        
        target_col = self.df.columns[-1]
        feature_cols = [col for col in self.df.columns if col != target_col]
        
        X = self.df[feature_cols]
        y = self.df[target_col]
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)
        
        # Encontrar número óptimo automáticamente
        optimal_n, feature_counts, cv_scores = self.find_optimal_features(X, y)
        
        selected_features_dict = {}
        
        # Método univariado con número óptimo
        print(f"\n🔹 Selección Univariada (k={optimal_n})...")
        selector_f = SelectKBest(score_func=f_classif, k=optimal_n)
        selector_f.fit(X, y_encoded)
        selected_f = X.columns[selector_f.get_support()].tolist()
        
        # Método RFE con número óptimo
        print(f"🔹 Recursive Feature Elimination (k={optimal_n})...")
        rf_estimator = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
        selector_rfe = RFE(estimator=rf_estimator, n_features_to_select=optimal_n)
        selector_rfe.fit(X, y_encoded)
        selected_rfe = X.columns[selector_rfe.get_support()].tolist()
        
        # Importancia Random Forest
        print(f"🔹 Feature Importance (k={optimal_n})...")
        rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        rf.fit(X, y_encoded)
        feature_importance = list(zip(X.columns, rf.feature_importances_))
        feature_importance.sort(key=lambda x: x[1], reverse=True)
        selected_importance = [feat for feat, imp in feature_importance[:optimal_n]]
        
        selected_features_dict = {
            'univariate': selected_f,
            'rfe': selected_rfe,
            'importance': selected_importance
        }
        
        if method == 'hybrid':
            # Combinar métodos - características que aparecen en al menos 2 métodos
            all_features = set()
            for features in selected_features_dict.values():
                all_features.update(features)
            
            feature_counts = {}
            for features in selected_features_dict.values():
                for feat in features:
                    feature_counts[feat] = feature_counts.get(feat, 0) + 1
            
            # Seleccionar características más consensuadas
            final_features = [feat for feat, count in feature_counts.items() if count >= 2]
            
            # Si no hay suficientes, tomar las más frecuentes
            if len(final_features) < optimal_n // 2:
                sorted_features = sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)
                final_features = [feat for feat, count in sorted_features[:optimal_n]]
            
            self.selected_features = final_features
        else:
            self.selected_features = selected_features_dict[method]
        
        print(f"\n✅ Características seleccionadas: {len(self.selected_features)}")
        print("🏆 Top 10 características:")
        for i, feat in enumerate(self.selected_features[:10], 1):
            print(f"  {i:2d}. {feat}")
        
        return self.selected_features, optimal_n, cv_scores
    
    def save_selected_features(self, filepath="selected_features.txt"):
        """Guardar características seleccionadas"""
        if self.selected_features is None:
            print("⚠️ No hay características seleccionadas para guardar.")
            return
        
        with open(filepath, "w") as f:
            f.write("Características seleccionadas:\n")
            f.write("=" * 30 + "\n")
            for i, feat in enumerate(self.selected_features, 1):
                f.write(f"{i:2d}. {feat}\n")
            f.write(f"\nTotal: {len(self.selected_features)} características")
        
        print(f"💾 Características guardadas en '{filepath}'")
    
    def run_feature_selection_analysis(self, df):
        """Ejecutar solo análisis de selección de características"""
        print("🚀 ANÁLISIS DE SELECCIÓN DE CARACTERÍSTICAS CICIDS 2018")
        print("=" * 60)
        
        # 1. Cargar y preparar datos
        self.load_data(df)
        
        # 2. Análisis exploratorio
        class_dist = self.exploratory_analysis()
        
        # 3. Selección automática de características
        selected_features, optimal_n, cv_scores = self.feature_selection(method='hybrid')
        
        # 4. Guardar características seleccionadas
        self.save_selected_features("selected_features_auto.txt")
        
        print("\n✅ ANÁLISIS COMPLETADO!")
        print(f"   📁 Archivo guardado: selected_features_auto.txt")
        print(f"   🔢 Características seleccionadas: {len(selected_features)}")
        print(f"   🎯 Número óptimo encontrado: {optimal_n}")
        
        return {
            'selected_features': selected_features,
            'optimal_n_features': optimal_n,
            'cv_scores': cv_scores,
            'class_distribution': class_dist
        }


# EJECUCIÓN PRINCIPAL
# Ruta de los CSV
ruta_carpeta = r"C:/U Octavo Semestre/tesis/Preprocesamiento de datos dataset actualizado/dataset_undersampling/"
archivos = [os.path.join(ruta_carpeta, f) for f in os.listdir(ruta_carpeta) if f.endswith(".csv")]

# Columnas a eliminar
exclude_cols = ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp', 'Bwd URG Flags']

print("📥 CARGANDO DATOS...")
# Cargar y concatenar todos los CSV en un solo DataFrame
dfs = []
for archivo in archivos:
    print(f"   Cargando: {os.path.basename(archivo)}")
    df_chunk = pl.read_csv(archivo)
    
    # Eliminar columnas no deseadas si existen
    cols_to_drop = [c for c in exclude_cols if c in df_chunk.columns]
    if cols_to_drop:
        df_chunk = df_chunk.drop(cols_to_drop)
    
    # Convertir todas las columnas a float64 excepto Label y Protocol
    for c in df_chunk.columns:
        if c not in ["Label", "Protocol"]:
            df_chunk = df_chunk.with_columns(pl.col(c).cast(pl.Float64))
    
    dfs.append(df_chunk)

# Concatenar todos los CSV
df = pl.concat(dfs, rechunk=True)
print(f"✅ Dataset cargado: {df.shape}")

# Normalizar etiquetas
label_mapping = {
    "Botnet Ares": "Bot",
    "Web Attack - Brute Force": "WebAttacks",
    "Web Attack - XSS": "WebAttacks",
    "Infiltration - Communication Victim Attacker": "Infilteration",
    "Infiltration - Dropbox Download": "Infilteration",
    "Infiltration - NMAP Portscan": "Infilteration",
}

df = df.with_columns([
    pl.when(pl.col("Label").str.to_lowercase() == "benign")
      .then(pl.lit("BENIGN"))
      .when(pl.col("Label").str.contains("Attempted"))
      .then(pl.lit("BENIGN"))
      .when(pl.col("Label").is_in(list(label_mapping.keys())))
      .then(
          pl.col("Label").replace_strict(
              label_mapping, 
              return_dtype=pl.Utf8,
              default="Other"
          )
      )
      .otherwise(pl.col("Label"))
      .alias("Label")
])

# Eliminar etiquetas con "SQL"
df = df.filter(~pl.col("Label").str.contains("SQL"))

print(f"📊 Distribución final de etiquetas:")
print(df['Label'].value_counts())

# Convertir a Pandas para el análisis
df = df.to_pandas()
print(f"✅ Dataset final preparado: {df.shape}")

# EJECUTAR ANÁLISIS DE CARACTERÍSTICAS
selector = CICIDSFeatureSelector()
results = selector.run_feature_selection_analysis(df=df)

📥 CARGANDO DATOS...
   Cargando: Friday-02-03-2018.csv
   Cargando: Friday-23-02-2018.csv
   Cargando: Thursday-01-03-2018.csv
   Cargando: Thursday-22-02-2018.csv
   Cargando: Wednesday-28-02-2018.csv
✅ Dataset cargado: (733312, 83)
📊 Distribución final de etiquetas:
shape: (4, 2)
┌───────────────┬────────┐
│ Label         ┆ count  │
│ ---           ┆ ---    │
│ str           ┆ u32    │
╞═══════════════╪════════╡
│ BENIGN        ┆ 500445 │
│ WebAttacks    ┆ 244    │
│ Bot           ┆ 142921 │
│ Infilteration ┆ 89663  │
└───────────────┴────────┘
✅ Dataset final preparado: (733273, 83)
🚀 ANÁLISIS DE SELECCIÓN DE CARACTERÍSTICAS CICIDS 2018
✓ Columna 'Label' movida al final como target
✓ Label Encoding aplicado a columna 'Protocol'
  Protocolos únicos codificados: 4
✓ Datos cargados: (733273, 83)
✓ Distribución de clases:
Label
BENIGN           500445
Bot              142921
Infilteration     89663
WebAttacks          244
Name: count, dtype: int64

📊 ANÁLISIS EXPLORATORIO
Shape: (733273